# Analysis of Layout ID

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1nkdkx2rI6nVKgoKBgkCUtfwEwuv8kptrRUXcXtfv0NM/edit?gid=247168398#gid=247168398]
- (Hypothesis for Klemmenbelegung)[https://docs.google.com/spreadsheets/d/1TSTxMCgEvuoayzOfx1MUqlV0tiqsVTBRN8aldlnFXxA/edit?gid=0#gid=0]

# import data

In [1]:
#imports
import os
import pandas as pd
import json
import ast
import re
from pprint import pprint

In [2]:
#build df from database exctracted data
filename='Building_device_Layout2025-07-14.csv'
extraced_date=filename[-14:-4]
filepath = os.path.join('INs', filename)
df = pd.read_csv(filepath)
#inspest column
df.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'postal_code',
       'city', 'coordinates', 'LayoutID', 'device_type', 'device_id',
       'modular_system', 'gfid'],
      dtype='object')

In [3]:
#filter out uncesseray infos
df=df[['building_id','customerID', 'customer_name','device_type', 'customer_name','address','LayoutID','modular_system']]

In [4]:
df.head()

,building_id,customerID,customer_name,device_type,customer_name,address,LayoutID,modular_system
0,756,66,HwS,RUT956,HwS,Wolzogenstr.28,gas:1--gas:2--heating-circuit:1--heating-circu...,"{'name': 'modular-system-entry', 'id': 'modula..."
1,757,66,HwS,RUT956,HwS,Hagelberger Str. 26,gas:1--gas:2--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
2,758,66,HwS,RUT956,HwS,Hochstr. 8,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
3,759,66,HwS,RUT956,HwS,Planufer 82a,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
4,761,66,HwS,RUT956,HwS,Gabainstr. 13,gas:1--gas:2--global-separation-circuit:1--hea...,"{'name': 'modular-system-entry', 'id': 'modula..."


In [5]:
df.count()

building_id       2014
customerID        2014
customer_name     2014
device_type       1086
customer_name     2014
address           2014
LayoutID          1138
modular_system    2014
dtype: int64

# Group by LayoutID

In [6]:
unique_count = df['LayoutID'].nunique()
print(f"Number of unique LayoutID values: {unique_count}")

Number of unique LayoutID values: 233


In [7]:
# Group by LayoutID
grouped = df.groupby('LayoutID').agg(
    Occurrence=('LayoutID', 'count'),
    buildingIDs=('building_id', list)
).reset_index()

# Add a simple index column starting from 0
grouped = grouped.sort_values(by='Occurrence', ascending=False).reset_index(drop=True)
grouped['KBn']=grouped.index+1
#grouped['KBn'] = grouped['KBn'].fillna(-1).astype(int)

In [8]:
# Show result
grouped.head(20)

,LayoutID,Occurrence,buildingIDs,KBn
0,gas:1--heating-circuit:1--warm-water:1,129,"[768, 769, 770, 778, 787, 1924, 1926, 1927, 46...",1
1,district-heating:1--heating-circuit:1--warm-wa...,84,"[774, 1909, 1910, 1911, 1912, 1913, 1314, 1315...",2
2,gas:1--heating-circuit:1,77,"[758, 759, 499, 1525, 1649, 1654, 1657, 1791, ...",3
3,gas:1--heat-exchanger:1--heating-circuit:1--wa...,46,"[1925, 1931, 1781, 1509, 1526, 1527, 1531, 153...",4
4,local-heating-station:1--heating-circuit:1--wa...,45,"[509, 790, 1005, 1628, 1629, 1630, 1631, 672, ...",5
5,gas:1--heat-exchanger:1--heating-circuit:1,40,"[1524, 1795, 1560, 1638, 1639, 604, 617, 623, ...",6
6,gas:1--heating-circuit:1--heating-circuit:2--w...,34,"[2219, 1511, 1515, 1516, 1517, 1518, 1537, 179...",7
7,gas:1--gas:2--heat-exchanger:1--heating-circui...,26,"[510, 1773, 1777, 1778, 1528, 1529, 1530, 1539...",8
8,district-heating:1--heating-circuit:1,22,"[762, 766, 777, 712, 450, 389, 391, 1725, 1726...",9
9,district-heating:1--heating-circuit:1--heating...,22,"[600, 1488, 1724, 796, 1772, 1385, 1386, 1389,...",10


In [9]:
# Sum the number of occurrences for top 10 and top 20
top_10_sum = grouped.head(10)['Occurrence'].sum()
top_20_sum = grouped.head(20)['Occurrence'].sum()
print(f"total LayoutID: {df.LayoutID.count()}")
print(f"top 10 LayoutIDs count: {top_10_sum}")
print(f"top 20 LayoutIDs count: {top_20_sum}")

print(f"Coverage top 10 LayoutIDs: {100*top_10_sum/df.LayoutID.count(): .1f} %")
print(f"Coverage top 20 LayoutIDs: {100*top_20_sum/df.LayoutID.count(): .1f} %")

total LayoutID: 1138
top 10 LayoutIDs count: 525
top 20 LayoutIDs count: 657
Coverage top 10 LayoutIDs:  46.1 %
Coverage top 20 LayoutIDs:  57.7 %


# new standard

In [10]:
# preparation add KBns
df = df.merge(grouped[['LayoutID', 'KBn']], on='LayoutID', how='left')
cols = ['KBn'] + [col for col in df.columns if col != 'KBn']
df = df[cols]
df.head()

,KBn,building_id,customerID,customer_name,customer_name,device_type,customer_name,customer_name,address,LayoutID,modular_system
0,46.0,756,66,HwS,HwS,RUT956,HwS,HwS,Wolzogenstr.28,gas:1--gas:2--heating-circuit:1--heating-circu...,"{'name': 'modular-system-entry', 'id': 'modula..."
1,34.0,757,66,HwS,HwS,RUT956,HwS,HwS,Hagelberger Str. 26,gas:1--gas:2--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
2,3.0,758,66,HwS,HwS,RUT956,HwS,HwS,Hochstr. 8,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
3,3.0,759,66,HwS,HwS,RUT956,HwS,HwS,Planufer 82a,gas:1--heating-circuit:1,"{'name': 'modular-system-entry', 'id': 'modula..."
4,32.0,761,66,HwS,HwS,RUT956,HwS,HwS,Gabainstr. 13,gas:1--gas:2--global-separation-circuit:1--hea...,"{'name': 'modular-system-entry', 'id': 'modula..."


In [11]:
# remove HX and GSC and combine HX with warmwater

In [12]:
def transform_to_tops(layout):
    if pd.isna(layout):
        return layout

    parts = layout.split('--')

    new_parts = []
    skip_next = False

    for i in range(len(parts)):
        if skip_next:
            skip_next = False
            continue

        part = parts[i]

        # Rule 1: heat-exchanger directly before warm-water → skip heat-exchanger, modify warm-water
        if re.match(r'heat-exchanger:\d+', part):
            if i + 1 < len(parts) and re.match(r'warm-water:(\d+)', parts[i + 1]):
                match = re.match(r'warm-water:(\d+)', parts[i + 1])
                new_parts.append(f"warm-water_external:{match.group(1)}")
                skip_next = True
            # Else → skip this heat-exchanger entirely
            continue

        # Rule 2: remove global-separation-circuit:<n>
        elif re.match(r'global-separation-circuit:\d+', part):
            continue

        # Rule 3: remove hydraulic-separator:<n>
        elif re.match(r'hydraulic-separator:\d+', part):
            continue

        else:
            new_parts.append(part)

    return '--'.join(new_parts)

In [13]:
# Apply to your DataFrame
df['LayoutID_Tops'] = df['LayoutID'].apply(transform_to_tops)

## get new sensor count

In [32]:
sensor_count_subsystem={'buffer-tank': 1,
 'chp': 2,
 'district-heating': 2,
 'gas': 2,
 'heat-pump': 2,
 'heating-circuit': 2,
 'local-heating-station': 2,
 'solar-thermal': 2,
 'warm-water': 4}

def calculate_sensor_count(layout):
    if pd.isna(layout):
        return 0

    total = 0
    parts = layout.split('--')
    for part in parts:
        subsystem = part.split(':')[0]
        # Remove suffixes like _mixed_ctrl or _external if present
        subsystem_clean = subsystem.split('_')[0]
        count = sensor_count_subsystem.get(subsystem_clean, 0)
        total += count
    return total

In [34]:
# Apply the function
df['new_sensor_count'] = df['LayoutID_Tops'].apply(calculate_sensor_count)

## group by LayoutID_Tops

In [36]:
unique_count = df['LayoutID_Tops'].nunique()
print(f"Number of unique LayoutID values: {unique_count}")

Number of unique LayoutID values: 195


In [38]:
# Group by LayoutID
grouped_new_std = df.groupby('LayoutID_Tops').agg(
    Occurrence=('LayoutID_Tops', 'count'),
    buildingIDs=('building_id', list),
    new_sensor_count=('new_sensor_count', set),
    Variations=('LayoutID', set),   
    Variations_KBn=('KBn', set)
).reset_index()

# Add a simple index column starting from 0
grouped_new_std = grouped_new_std.sort_values(by='Occurrence', ascending=False).reset_index(drop=True)
grouped_new_std=grouped_new_std[['LayoutID_Tops','Occurrence','new_sensor_count','Variations_KBn','buildingIDs','Variations']]

grouped_new_std['Variations_KBn'] = grouped_new_std['Variations_KBn'].apply(
    lambda s: ', '.join(f"KB{int(x)}" for x in sorted(s))
)

In [39]:
grouped_new_std.head(20)

,LayoutID_Tops,Occurrence,new_sensor_count,Variations_KBn,buildingIDs,Variations
0,gas:1--heating-circuit:1--warm-water:1,187,{8},"KB1, KB4, KB15","[768, 769, 770, 778, 787, 304, 1924, 1925, 192...",{gas:1--heat-exchanger:1--heating-circuit:1--w...
1,gas:1--heating-circuit:1,132,{4},"KB3, KB6, KB14","[758, 759, 499, 507, 1524, 1525, 1649, 1654, 1...","{gas:1--heat-exchanger:1--heating-circuit:1, g..."
2,district-heating:1--heating-circuit:1--warm-wa...,84,{8},KB2,"[774, 1909, 1910, 1911, 1912, 1913, 1314, 1315...",{district-heating:1--heating-circuit:1--warm-w...
3,gas:1--heating-circuit:1--heating-circuit:2--w...,51,{10},"KB7, KB23, KB37, KB197","[1775, 1779, 2219, 1511, 1515, 1516, 1517, 151...",{gas:1--heat-exchanger:1--heating-circuit:1--h...
4,local-heating-station:1--heating-circuit:1--wa...,45,{8},KB5,"[509, 790, 1005, 1628, 1629, 1630, 1631, 672, ...",{local-heating-station:1--heating-circuit:1--w...
5,gas:1--gas:2--heating-circuit:1--warm-water:1,40,{10},"KB8, KB19, KB62","[478, 510, 1773, 1777, 1778, 1528, 1529, 1530,...",{gas:1--gas:2--global-separation-circuit:1--he...
6,gas:1--gas:2--heating-circuit:1,31,{6},"KB12, KB29, KB34","[757, 773, 429, 1506, 668, 680, 696, 745, 530,...",{gas:1--gas:2--heat-exchanger:1--heating-circu...
7,gas:1--gas:2--heating-circuit:1--heating-circu...,24,{12},"KB22, KB26, KB50","[1928, 1930, 506, 333, 417, 388, 1641, 414, 79...",{gas:1--gas:2--heat-exchanger:1--heating-circu...
8,district-heating:1--heating-circuit:1,22,{4},KB9,"[762, 766, 777, 712, 450, 389, 391, 1725, 1726...",{district-heating:1--heating-circuit:1}
9,district-heating:1--heating-circuit:1--heating...,22,{10},KB10,"[600, 1488, 1724, 796, 1772, 1385, 1386, 1389,...",{district-heating:1--heating-circuit:1--heatin...


In [18]:
# Sum the number of occurrences for top 10 and top 20
top_10_sum = grouped_new_std.head(10)['Occurrence'].sum()
top_20_sum = grouped_new_std.head(20)['Occurrence'].sum()
print(f"total LayoutID: {df.LayoutID.count()}")
print(f"top 10 LayoutIDs count: {top_10_sum}")
print(f"top 20 LayoutIDs count: {top_20_sum}")

print(f"Coverage top 10 LayoutIDs: {100*top_10_sum/df.LayoutID.count(): .1f} %")
print(f"Coverage top 20 LayoutIDs: {100*top_20_sum/df.LayoutID.count(): .1f} %")

total LayoutID: 1138
top 10 LayoutIDs count: 638
top 20 LayoutIDs count: 782
Coverage top 10 LayoutIDs:  56.1 %
Coverage top 20 LayoutIDs:  68.7 %


## explore modular_system

In [19]:
if True: #test single buildings
    building_id = 1580#2389#1801#2317#1315#1593#1809 #1801 #
    mod_sys = df.loc[df['building_id'] == building_id, 'modular_system'].iloc[0]
    modular_system = ast.literal_eval(mod_sys)
    print(df.loc[df['building_id'] == building_id, 'LayoutID_Tops'].iloc[0])
    pprint(modular_system)

gas:1--heating-circuit:1
{'buildingID': 1580,
 'id': 'modular-system--osningstr-16A',
 'layoutID': 'gas:1--global-separation-circuit:1--heating-circuit:1',
 'name': 'modular-system-entry',
 'published': True,
 'publishedAt': '2024-12-02T11:59:22.665Z',
 'subSystems': [{'connections': [{'from': {'direction': 'bottom',
                                           'id': 'gas:1',
                                           'subSystem': {'hydraulicLocationIndex': 1,
                                                         'name': 'gas',
                                                         'side': 'right'}},
                                  'status': 'hot'},
                                 {'status': 'cold',
                                  'to': {'direction': 'bottom',
                                         'id': 'gas:1',
                                         'subSystem': {'hydraulicLocationIndex': 1,
                                                       'name': 'gas',
           

In [20]:
#buildings_with_unmixed_unctrl = df[df['new_LayoutID'].str.contains('heating-circuit_unmixed_unctrl', na=False)]

# Display the result
#print(buildings_with_unmixed_unctrl[['buildingID', 'new_LayoutID']])

## get subcomponent

In [21]:
def extract_unique_subcomponents(df, column='modular_system'):
    subcomponent_set = set()

    for entry in df[column]:
        try:
            # Safely convert string representation of dict to actual dict
            data = ast.literal_eval(entry)
        except Exception as e:
            print(f"Skipping invalid entry due to error: {e}")
            continue

        sub_systems = data.get("subSystems", [])
        for subsystem in sub_systems:
            parameters = subsystem.get("parameters", {})
            sub_components = parameters.get("subComponent", {})
            
            if isinstance(sub_components, dict):
                for key in sub_components.keys():
                    # Remove trailing --<number>
                    base_key = re.sub(r'--\d+$', '', key)

                    # Check prefix
                    if base_key.startswith(("district-heating--", "local-heating-station--", "heating-circuit--")):
                        subcomponent_set.add(base_key)

    return sorted(subcomponent_set)

In [22]:
unique_subcomponents = extract_unique_subcomponents(df)
for compo in unique_subcomponents:
    print(compo)

district-heating--heat-exchanger
district-heating--pump-sec
district-heating--sec-flow-temp
district-heating--valve
heating-circuit--placeholder-cold-connection-node
heating-circuit--pump
heating-circuit--secondary-flow-temp
heating-circuit--valve
local-heating-station--heat-exchanger
local-heating-station--pump-prim
local-heating-station--pump-sec


## get subSystems

In [23]:
# Sample: df['LayoutID_Tops'] = your actual column
def extract_words(layout_str):
    # Split by '--', then split each part by ':'
    parts = layout_str.split('--')
    words = []
    for part in parts:
        words.extend(part.split(':'))
    return words

# Apply and flatten the list
all_words = df['LayoutID'].dropna().apply(extract_words).explode()

# Get unique values
unique_words = all_words.unique().tolist()

# Optional: sort alphabetically or numerically
unique_words.sort()

#get rid of pipes and numbers
cleaned_words = [
    word for word in unique_words
    if not word.isdigit() and
    'pipe' not in word.lower() and
    'valve' not in word.lower()
]

In [24]:
for word in cleaned_words:
    print(word)#cleaned_words

1-heating-circuit
buffer-tank
chp
district-heating
gas
global-separation-circuit
heat-exchanger
heat-pump
heating-circuit
hydraulic-separator
local-heating-station
solar-thermal
warm-water


In [25]:
cleaned_words

['1-heating-circuit',
 'buffer-tank',
 'chp',
 'district-heating',
 'gas',
 'global-separation-circuit',
 'heat-exchanger',
 'heat-pump',
 'heating-circuit',
 'hydraulic-separator',
 'local-heating-station',
 'solar-thermal',
 'warm-water']

# exports

In [26]:
if True:
    filename1='Building_ID_VS_LayoutID_'+extraced_date+'.csv'
    filename2='LayoutID_occurence_'+extraced_date+'.csv'
    filename3='LayoutID_occurence_new_std_'+extraced_date+'.csv'
    filepath1 = os.path.join('OUTs', filename1)
    filepath2 = os.path.join('OUTs', filename2)
    filepath3 = os.path.join('OUTs', filename3)
    df.to_csv(filepath1, index=False)
    grouped.to_csv(filepath2, index=False)
    grouped_new_std.to_csv(filepath3, index=False)